## Импорт

In [1]:
import os
import re
import string
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import nltk
from nltk.corpus import stopwords
import pandas as pd
import json


## Подготовка данных

In [2]:
# Пути к данным
raw_data_dir = '../data/raw/'
output_path = '../data/processed/context_answer.csv'
character = 'House'


def clean_text(text):
    '''Функция для очистки текста'''
    text = re.sub(r'\[.*?\]|\(.*?\)', '', text)  # Удаление текста в скобках 
    text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
    text = text.strip()  # Удаление пробелов в начале и конце
    # text = text.lower()  # Приведение к нижнему регистру (опционально)
    # text = text.translate(str.maketrans('', '', string.punctuation))  # Удаление пунктуации (опционально)
    return text

# Загрузка всех CSV-файлов из директории
all_files = [os.path.join(raw_data_dir, f) for f in os.listdir(raw_data_dir) if f.endswith('.csv')]
df_list = []
for file in all_files:
    try:
        df = pd.read_csv(file, encoding='ISO-8859-1')
        df_list.append(df)
    except UnicodeDecodeError:
        df = pd.read_csv(file, encoding='utf-8')
        df_list.append(df)
full_text = pd.concat(df_list, ignore_index=True)

# Очистка данных
full_clean_text = full_text.dropna(subset=['name', 'line']).reset_index(drop=True)
full_clean_text.loc[:, 'line'] = full_clean_text['line'].apply(clean_text)

# Формирование пар "контекст-ответ"
min_length = 5  # Минимальная длина реплики
pairs = []
for i in range(1, len(full_clean_text)):
    if full_clean_text.loc[i, 'name'] == character:
        context = full_clean_text.loc[i - 1, 'line']
        response = full_clean_text.loc[i, 'line']
        if context and response and len(context.split()) >= min_length and len(response.split()) >= min_length:
            pairs.append({'context': context, 'response': response})
pairs_df = pd.DataFrame(pairs)


## Анализ и очистка данных

In [3]:
# Основная информация
print("Основная информация о данных:")
print(pairs_df.info())

# Проверка на пропуски
print("\nПропуски в данных:")
print(pairs_df.isnull().sum())

# Добавим столбцы с длиной реплик
pairs_df['context_length'] = pairs_df['context'].apply(lambda x: len(x.split()))
pairs_df['response_length'] = pairs_df['response'].apply(lambda x: len(x.split()))

# Поиск и удаление аномалий
allowed_chars = r"[a-zA-Z0-9\s.,!?;:'\"()\-]"  # Определим допустимые символы с помощью регулярного выражения
anomalous_chars = set()


def find_anomalous_characters(text):
    anomalous_chars = set()
    for char in text:
        if not re.match(allowed_chars, char):
            anomalous_chars.add(char)
    return anomalous_chars

for text in pairs_df['context']:
    anomalous_chars.update(find_anomalous_characters(text))
for text in pairs_df['response']:
    anomalous_chars.update(find_anomalous_characters(text))

print("\nНайденные аномальные символы:", anomalous_chars)


def remove_anomalous_characters(text):
    """Функция для удаления запрещенных символов"""
    # Используем регулярное выражение для поиска всех допустимых символов
    cleaned_text = re.findall(allowed_chars, text)
    # Соединяем найденные символы обратно в строку
    return ''.join(cleaned_text)

# Применим функцию ко всем репликам в DataFrame
pairs_df['context'] = pairs_df['context'].apply(remove_anomalous_characters)
pairs_df['response'] = pairs_df['response'].apply(remove_anomalous_characters)

# Настройка стиля графиков
plt.style.use('seaborn')
colors = ['#3498db', '#e74c3c', '#2ecc71']  # Синий, красный, зеленый

### 1. Совмещенное распределение длин реплик с KDE ###
plt.figure(figsize=(12, 6))

# Гистограмма с KDE
sns.histplot(data=pairs_df, x='context_length', color=colors[0], 
             label='Контекст', kde=True, alpha=0.5, bins=30)
sns.histplot(data=pairs_df, x='response_length', color=colors[1], 
             label='Ответ', kde=True, alpha=0.5, bins=30)

# Вертикальные линии средних значений
mean_ctx = pairs_df['context_length'].mean()
mean_resp = pairs_df['response_length'].mean()
plt.axvline(mean_ctx, color=colors[0], linestyle='--', linewidth=2)
plt.axvline(mean_resp, color=colors[1], linestyle='--', linewidth=2)

# Аннотации
plt.text(mean_ctx+2, plt.ylim()[1]*0.8, 
         f'Среднее: {mean_ctx:.1f}', color=colors[0], fontsize=12)
plt.text(mean_resp+2, plt.ylim()[1]*0.7, 
         f'Среднее: {mean_resp:.1f}', color=colors[1], fontsize=12)

plt.title('Совмещенное распределение длин реплик', fontsize=14)
plt.xlabel('Длина в словах', fontsize=12)
plt.ylabel('Частота', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 2. Топ-слов в виде горизонтальных барчартов ###
def plot_top_words(word_counts, title, color):
    words, counts = zip(*word_counts)
    plt.figure(figsize=(12, 6))
    bars = plt.barh(words[::-1], counts[::-1], color=color, alpha=0.7)  # reverse для сортировки
    
    # Аннотации значений
    for bar in bars:
        width = bar.get_width()
        plt.text(width + 5, bar.get_y() + bar.get_height()/2, 
                 f'{width}', va='center', fontsize=10)
    
    plt.title(title, fontsize=14)
    plt.xlabel('Количество употреблений', fontsize=12)
    plt.grid(True, axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()

plot_top_words(top_context_words, 'Топ-20 слов в контекстах', colors[0])
plot_top_words(top_response_words, 'Топ-20 слов в ответах', colors[1])

### 3. Круговая диаграмма уникальности ###
unique_stats = {
    'Уникальные контексты': unique_contexts,
    'Уникальные ответы': unique_responses,
    'Дубликаты пар': duplicates
}

plt.figure(figsize=(10, 10))
values = list(unique_stats.values())
labels = [f'{k}\n({v} | {v/sum(values)*100:.1f}%)' for k, v in unique_stats.items()]

plt.pie(values, labels=labels, colors=colors, autopct='', 
        startangle=90, shadow=True, explode=(0.1, 0, 0))
plt.title('Распределение уникальности данных', fontsize=14)
plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1))
plt.tight_layout()
plt.show()

### 4. Ящики с усами для длин ###
plt.figure(figsize=(12, 6))
sns.boxplot(data=pairs_df[['context_length', 'response_length']], 
            palette=colors[:2], showfliers=False)

# Аннотации медиан
medians = pairs_df[['context_length', 'response_length']].median().values
for xtick in plt.xticks()[0]:
    plt.text(xtick, medians[xtick]+1, f'Медиана: {medians[xtick]:.1f}', 
             ha='center', color=colors[xtick], fontsize=12)

plt.title('Распределение длин реплик (без выбросов)', fontsize=14)
plt.ylabel('Длина в словах', fontsize=12)
plt.xticks([0, 1], ['Контексты', 'Ответы'], fontsize=12)
plt.grid(True, axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 5. Скаттерплот длин контекста и ответа ###
plt.figure(figsize=(12, 6))
scatter = sns.regplot(x='context_length', y='response_length', 
                     data=pairs_df, color=colors[2], 
                     scatter_kws={'alpha':0.3}, line_kws={'color':'red'})

# Расчет корреляции
corr = pairs_df[['context_length', 'response_length']].corr().iloc[0,1]
plt.text(0.95, 0.95, f'Pearson R: {corr:.2f}', 
         transform=plt.gca().transAxes, ha='right', 
         fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

plt.title('Зависимость длины ответа от контекста', fontsize=14)
plt.xlabel('Длина контекста', fontsize=12)
plt.ylabel('Длина ответа', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

def get_top_words(column, top_n=20):
    """Топ-20 слов в контекстах и ответах"""
    all_words = ' '.join(column).split()
    filtered_words = [word for word in all_words if word.lower() if len(word) > 3]
    word_counts = Counter(filtered_words)
    return word_counts.most_common(top_n)

top_context_words = get_top_words(pairs_df['context'])
print("\nТоп-20 слов в контекстах:", top_context_words)

top_response_words = get_top_words(pairs_df['response'])
print("Топ-20 слов в ответах:", top_response_words)

# Уникальные реплики
unique_contexts = pairs_df['context'].nunique()
unique_responses = pairs_df['response'].nunique()

print(f"\nУникальных контекстов: {unique_contexts}")
print(f"Уникальных ответов: {unique_responses}")

# Поиск дубликатов
duplicates = pairs_df.duplicated(subset=['context', 'response']).sum()
print(f"\nКоличество дубликатов пар 'контекст-ответ': {duplicates}")

# Случайные примеры
print("\nСлучайные примеры пар 'контекст-ответ':")
print(pairs_df.sample(5))

# Сохранение данных
pairs_df.to_csv(output_path, index=False)


Основная информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12329 entries, 0 to 12328
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   context   12329 non-null  object
 1   response  12329 non-null  object
dtypes: object(2)
memory usage: 192.8+ KB
None

Пропуски в данных:
context     0
response    0
dtype: int64

Найденные аномальные символы: {'+', '/', '\x97', '&', 'º', '@', '%', '*', '\x9f', '¡', '±', '\x89', 'é', '_', '\x9d', '$', '{', '§', 'ï', '¿', 'Ã', '#', '\x93', ']', '½', '³', '¢', '¯', '[', '¶'}


OSError: 'seaborn' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

## Подготовка данных для GPT2

In [ ]:
# Загрузка данных из CSV
data_path = '../data/processed/context_answer.csv'
df = pd.read_csv(data_path)

# Преобразование данных в JSON
data_json = []

for index, row in df.iterrows():
    # Создаем запись с полями character, q и a
    entry = {
        "character": "house",  # Укажите имя персонажа
        "q": row["context"],      # context -> q
        "a": row["response"]      # response -> a
    }
    data_json.append(entry)
    

## Подготовка данных для LLAMA

In [ ]:
# Сохранение в JSON-файл
output_path = '../data/processed/context_answer.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(data_json, f, ensure_ascii=False, indent=4)

print(f"Данные сохранены в {output_path}")
